In [ ]:
import pandas as pd 
from path_fns.filepaths import TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100

### Person data

In [ ]:
df_persons = pd.read_parquet(TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON)
len(df_persons)


In [ ]:
df_persons.columns

In [ ]:
df_persons.head(10)


In [ ]:

for col in ["humanLabel", "humanAltLabel", "birth_name", "given_nameLabel", "family_nameLabel"]:
    df_persons[col] = df_persons[col].map(lambda x: list(map(str.lower, x)))

df_persons.head(2)

In [ ]:
df_persons["occupationLabel"].head(1000).value_counts()

### Names

In [ ]:
from master_data.names import get_df_given_names_with_freqs, get_df_family_names_with_freqs
df_given_names = get_df_given_names_with_freqs(spark)
df_family_names = get_df_family_names_with_freqs(spark)
df_given_names.createOrReplaceTempView("df_given_names")
df_family_names.createOrReplaceTempView("df_family_names")
display(df_given_names.limit(2).toPandas())
display(df_family_names.limit(2).toPandas())

### Person postcode lookup

In [ ]:
from master_data.utils import get_person_nearby_postcodes_lookup
df_point_postcode = get_person_nearby_postcodes_lookup(spark)
df_point_postcode.createOrReplaceTempView("df_point_postcode")


df_point_postcode.limit(3).toPandas()

## Occupations

In [ ]:
occupations = spark.read.parquet("scrape_wikidata/raw_data/occupations/")
occupations.createOrReplaceTempView("occupations")
occupations.limit(2).toPandas()

sql = """
select human, array_distinct(collect_list(occupationLabel)) as occupation_options
from occupations
group by human
"""

occupations = spark.sql(sql)
occupations.createOrReplaceTempView("occupations")
occupations.limit(2).toPandas()

## Master data

#### Get list of given names and family names

In [ ]:
from master_data.names import get_df_filter, name_split
df_person = name_split(df_person, spark)
df_person.limit(2).toPandas()

In [ ]:
remove_dupe_country_citizen = """
array_distinct(
    transform(
        split(h.country_citizenlabel, ' \\\\| '), 
            x -> case 
                    when x = 'United Kingdom of Great Britain and Ireland' then 'United Kingdom' 
                    else x 
                    end
    )
)
"""

sql = f"""
select 
h.row_num,
h.human, 
h.humanlabel,
split(h.humanaltlabel, ", ") as humanaltlabel,
occ.occupation_options,
substr(h.dob,1,10) as dob, 


{remove_dupe_country_citizen}  as country_citizenship,


place_birthlabel as birth_place,
birth_countrylabel as birth_country,
sex_or_genderlabel as gender,



residencelabel as residence_place,
residence_countrylabel as residence_country,

pc.birthplace_nearby_locs,
pc.residence_nearby_locs,
pc.random_nearby_locs,

h.given_name_1 as given_name_1,
n1.alt_names as alt_given_name_1,
h.given_name_2 as given_name_2,
n2.alt_names as alt_given_name_2,
h.given_name_3 as given_name_3,
n3.alt_names as alt_given_name_3,
h.family_name_1 as family_name_1,
n4.alt_names as alt_family_name_1,
h.family_name_2 as family_name_2,
n5.alt_names as alt_family_name_2

from df_person as h

left join df_given_names as n1
on lower(h.given_name_1) = n1.original_name

left join df_given_names as n2
on lower(h.given_name_2) = n2.original_name

left join df_given_names as n3
on lower(h.given_name_3) = n3.original_name

left join df_family_names as n4
on lower(h.family_name_1) = n4.original_name

left join df_family_names as n5
on lower(h.family_name_2) = n5.original_name

left join df_point_postcode as pc
on h.human = pc.person

left join occupations  as occ
on h.human = occ.human



"""
df_final = spark.sql(sql)
df_final.createOrReplaceTempView("df_final")  


df_final.limit(10).toPandas()


## Now want to link parent back onto this table, and their location.

In [ ]:
df_family = spark.read.parquet("scrape_wikidata/raw_data/family/parent_child/")
 
df_one_parent = df_family.dropDuplicates(["child"])
# df_one_child = df_family.dropDuplicates(["human"])
df_one_parent.createOrReplaceTempView("df_one_parent") 
# df_one_child.createOrReplaceTempView("df_one_child") 
df_family.limit(2).toPandas()

In [ ]:
sql = """

select 
    df.*, 
    p.human as parent, 
    p_details.birthplace_nearby_locs as parent_birthplace_loc, 
    p_details.random_nearby_locs as parent_random_loc


from df_final as df
left join df_one_parent as p
on df.human = p.child
left join df_final as p_details
on p.human = p_details.human




"""
# No need for child as if parents take childs details and children take parents, then you don't achieve parents and children having the same address!
# left join df_one_child as c
# on df.human = c.human
# left join df_final as c_details
# on c.child = c_details.human
df_final_with_parents = spark.sql(sql)

In [ ]:
df_final_with_parents = df_final_with_parents.repartition(100)
df_final_with_parents.write.mode('overwrite').parquet("scrape_wikidata/clean_data/master_data/")

In [ ]:
import glob
import os 
files = sorted(
    glob.glob("scrape_wikidata/clean_data/master_data/*.parquet")
)

for i, f in enumerate(files):
    
    base = os.path.basename(f)
    f2 = f.replace(base, f"{i:03}.parquet")
    
    os.rename(f, f2)
    
files = sorted(
    glob.glob("scrape_wikidata/clean_data/master_data/.*crc")
)
for f in files:
    os.remove(f)

In [ ]:
spark.read.parquet("corrupted_data/uk_citizens_groupsize_20/").count()